In [1]:
# pip install jiwer

In [7]:
# !pip install transformers datasets jiwer soundfile accelerate

In [3]:
# !pip install onedrivesdk requests tqdm

In [4]:
# !pip install deep-translator

In [7]:
# Cell 2: Imports
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
# conda install -c pytorch torchaudio

import os
import json
import random
import pandas as pd
from pathlib import Path
import torchaudio
# from deep_translator import GoogleTranslator

In [8]:
!python --version

Python 3.12.9


In [2]:
!pip install "transformers[speech]"


In [2]:
# !pip install "transformers[speech]==4.52.4" --force-reinstall


In [ ]:
import sys, os, transformers, pkgutil, pathlib
print("Python:", sys.version)
print("transformers version:", transformers.__version__)

# Make sure you don't have a local file/folder named 'transformers' shadowing the package
import os, glob
print("CWD:", os.getcwd())
print("Local matches:", glob.glob("./transformers*"))


In [8]:
!pip install evaluate


In [ ]:
# import boto3

# # S3 details
# bucket = "whispersmalltest"
# key = "OneDrive_1_8-22-2025.zip"
# local_file = "OneDrive_1_8-22-2025.zip"

# # Download from S3 to local SageMaker Studio storage
# s3 = boto3.client("s3")
# s3.download_file(bucket, key, local_file)

# print("File downloaded:", local_file)


In [ ]:
# pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


In [9]:
# Update system packages
!pip install imageio[ffmpeg] ffmpeg-python


  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl (29.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ffmpeg-python]]


In [10]:
import imageio
# !pip install imageio-ffmpeg
import os
import json
import zipfile
import random

# ---- Step 2: Load JSON (outside zip) ----
json_file = "audio_transcripts_updated.json"  # change this if your JSON has another name

if not os.path.exists(json_file):
    raise FileNotFoundError(f"{json_file} not found outside the zip!")

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ---- Step 3: Shuffle & Split (80/10/10) ----
random.shuffle(data)
n = len(data)
train_end = int(0.8 * n)
valid_end = int(0.9 * n)

train_data = data[:train_end]
valid_data = data[train_end:valid_end]
test_data = data[valid_end:]

# ---- Step 4: Save in JSONL format ----
def save_jsonl(path, records):
    with open(path, "w", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

save_jsonl("train.jsonl", train_data)
save_jsonl("valid.jsonl", valid_data)
save_jsonl("test.jsonl", test_data)

print(f"✅ Saved train.jsonl ({len(train_data)} records), "
      f"valid.jsonl ({len(valid_data)} records), "
      f"test.jsonl ({len(test_data)} records))")

# ---- Step 5: Fix audio file paths ----
# def fix_paths(input_path, output_path, base_dir="data/"):
#     fixed_lines = []
#     with open(input_path, "r", encoding="utf-8") as f:
#         for line in f:
#             data = json.loads(line)
#             # Fix audio_filepath
#             filename = data["audio_filepath"].split("/")[-1]  # keep only filename
#             data["audio_filepath"] = base_dir + filename
#             fixed_lines.append(json.dumps(data))

#     # Write fixed JSONL
#     with open(output_path, "w", encoding="utf-8") as f:
#         for line in fixed_lines:
#             f.write(line + "\n")

# # Apply to train, valid, and test
# fix_paths("train.jsonl", "train_fixed.jsonl")
# fix_paths("valid.jsonl", "valid_fixed.jsonl")
# fix_paths("test.jsonl", "test_fixed.jsonl")

# print("✅ Fixed JSONL files created:")
# print("Train:", "train_fixed.jsonl")
# print("Valid:", "valid_fixed.jsonl")
# print("Test :", "test_fixed.jsonl")


✅ Saved train.jsonl (32 records), valid.jsonl (4 records), test.jsonl (4 records))


In [6]:
import transformers
print(transformers.__version__)


4.57.0


In [4]:
# Cell 2: Import libraries
# !pip install -U transformers datasets evaluate accelerate librosa jiwer soundfile

import os
import torch
# import torchaudio
import json
from datasets import load_dataset, Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, TrainingArguments, Trainer, Seq2SeqTrainer
from transformers.trainer_callback import TrainerCallback
import evaluate
from transformers import TrainingArguments
import numpy as np
from datasets import Dataset, load_dataset
import json

# Function to load JSONL into Python list
# def load_jsonl(path):
#     with open(path, "r", encoding="utf-8") as f:
#         return [json.loads(line) for line in f]

# train_path = "/home/sagemaker-user/train.jsonl"
# valid_path = "/home/sagemaker-user/valid.jsonl"
# Convert to Hugging Face Dataset
from datasets import load_dataset
# !pip install soundfile

train_dataset = load_dataset("json", data_files="./train.jsonl")["train"]
valid_dataset = load_dataset("json", data_files="./valid.jsonl")["train"]

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (/opt/conda/lib/python3.12/site-packages/transformers/__init__.py)

In [ ]:
# !pip install librosa
from transformers import WhisperForConditionalGeneration, WhisperProcessor


processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3-turbo")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3-turbo")
from datasets import Audio

# Tell HuggingFace to decode audio automatically
train_dataset = train_dataset.cast_column("audio_filepath", Audio(sampling_rate=16000))
valid_dataset = valid_dataset.cast_column("audio_filepath", Audio(sampling_rate=16000))
# !pip install torchcodec
def preprocess(example):
    # The dataset now gives you {"array": np.array, "sampling_rate": 16000}
    audio = example["audio_filepath"]
    input_features = processor.feature_extractor(audio["array"], sampling_rate=16000, return_tensors="pt").input_features[0]
    labels = processor.tokenizer(example["human_transcript"]).input_ids

    example["input_features"] = input_features
    example["labels"] = labels
    return example

train_dataset = train_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)


In [ ]:
def data_collator(features):
    # Ensure input_features are also converted to tensors
    input_features = torch.stack([torch.tensor(f["input_features"]) for f in features])
    label_ids = [f["labels"] for f in features]
    labels = processor.tokenizer.pad({"input_ids": label_ids}, return_tensors="pt").input_ids
    labels[labels == processor.tokenizer.pad_token_id] = -100
    return {"input_features": input_features, "labels": labels}
    wer_metric = evaluate.load("wer")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred_ids = np.argmax(logits, axis=-1)
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {"wer": wer_metric.compute(predictions=pred_str, references=label_str)}
# Cell 8: Trainer callback for logging
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Epoch {state.epoch:.2f} - Logs: {logs}")

In [6]:
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./fine_tuned_whisper",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    predict_with_generate=True  # this is important for WER calculation
)
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./fine_tuned_whisper",
#     per_device_train_batch_size=2,   # use 2 if your GPU allows
#     num_train_epochs=3,              # train for a few epochs
#     learning_rate=1e-4,              # slightly higher LR works well for Whisper
#     logging_dir="./logs",
#     logging_steps=100,
#     predict_with_generate=True       # needed for WER
# )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=processor,  # processor includes feature extractor + tokenizer
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_802/1004993879.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [7]:
# !pip install bitsandbytes
# !pip install peft 
from peft import get_peft_model, LoraConfig, TaskType

# LoRA Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["out_proj", "k_proj"],  # change to ["out_proj", "k_proj"] if CPU
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 7,864,320 || all params: 1,551,169,280 || trainable%: 0.5070


In [10]:
trainer.train()
# import torch
# torch.cuda.empty_cache()
import torch

# Clear cache
# torch.cuda.empty_cache()

# # Optional: reset memory stats
# torch.cuda.reset_peak_memory_stats()


Step,Training Loss
500,0.791800
1000,0.256800


/opt/conda/lib/python3.12/site-packages/transformers/modeling_utils.py:3852: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1062, training_loss=0.5081760178402543, metrics={'train_runtime': 4120.9522, 'train_samples_per_second': 0.515, 'train_steps_per_second': 0.258, 'total_flos': 4.5336687427584e+18, 'train_loss': 0.5081760178402543, 'epoch': 3.0})

In [11]:
# Cell 12: Save final model
model.save_pretrained("3epochND_largev2lora_adapter")
processor.save_pretrained("3epochND_largev2lora_fine_tuned_whisper")

[]

In [12]:
# !pip install librosa
# import json
# import torch
# import librosa
# import evaluate
# import pandas as pd
# from datasets import Dataset
# from transformers import WhisperProcessor, WhisperForConditionalGeneration

# # --------------------------------------------------
# # Load model and processor
# # --------------------------------------------------
# model = WhisperForConditionalGeneration.from_pretrained(
#     "1epoch_largev2lora_adapter"
# )
# processor = WhisperProcessor.from_pretrained(
#     "1epoch_largev2lora_fine_tuned_whisper"
# )

# # Disable forced decoder IDs
# model.generation_config.forced_decoder_ids = None
# model.generation_config.language = "en"   # force English
# model.eval()

# # --------------------------------------------------
# # Load WER metric
# # --------------------------------------------------
# wer_metric = evaluate.load("wer")

# # --------------------------------------------------
# # Load validation dataset
# # --------------------------------------------------
# def load_jsonl(path):
#     with open(path, "r") as f:
#         return [json.loads(line) for line in f]

# data_list = load_jsonl("test.jsonl")
# data_dict = {k: [d[k] for d in data_list] for k in data_list[0]}
# valid_dataset = Dataset.from_dict(data_dict)

# # --------------------------------------------------
# # Transcribe & Compare
# # --------------------------------------------------
# def transcribe_and_compare(dataset):
#     predictions, references, files = [], [], []

#     for example in dataset:
#         try:
#             # 🔹 Load & resample audio to 16kHz
#             waveform, sr = librosa.load(example["audio_filepath"], sr=16000)

#             inputs = processor(
#                 waveform, sampling_rate=16000, return_tensors="pt"
#             )
#             input_features = inputs.input_features

#             with torch.no_grad():
#                 predicted_ids = model.generate(input_features=input_features)

#             pred_text = processor.batch_decode(
#                 predicted_ids, skip_special_tokens=True
#             )[0].strip()

#             predictions.append(pred_text)
#             references.append(example["human_transcript"].strip())
#             files.append(example["audio_filepath"])

#         except Exception as e:
#             print(f"❌ Skipping {example['audio_filepath']}: {e}")

#     if not predictions:
#         print("⚠️ No valid transcriptions found.")
#         return

#     # Compute WER
#     wer = wer_metric.compute(predictions=predictions, references=references)
#     print(f"\n✅ Final Word Error Rate (WER): {wer:.3f}")

#     # Side-by-side comparison
#     df = pd.DataFrame({
#         "Audio File": files,
#         "Reference Transcript": references,
#         "Model Transcript": predictions
#     })

#     from IPython.display import display
#     display(df)

#     return df

# # --------------------------------------------------
# # Run transcription
# # --------------------------------------------------
# df = transcribe_and_compare(valid_dataset)



In [13]:
# # import json
# # import torch
# # import librosa
# # import evaluate
# # import pandas as pd
# # from datasets import Dataset
# # from transformers import WhisperProcessor, WhisperForConditionalGeneration

# # # --------------------------------------------------
# # # Load model and processor
# # # --------------------------------------------------
# # model = WhisperForConditionalGeneration.from_pretrained(
# #     "/home/sagemaker-user/1epoch_largelora_adapter"
# # )
# # processor = WhisperProcessor.from_pretrained(
# #     "/home/sagemaker-user/1epoch_largelora_fine_tuned_whisper"
# # )

# # # Disable forced decoder IDs
# # model.generation_config.forced_decoder_ids = None
# # model.generation_config.language = "en"   # force English
# # model.eval()

# # # --------------------------------------------------
# # # Load WER metric
# # # --------------------------------------------------
# # wer_metric = evaluate.load("wer")

# # # --------------------------------------------------
# # # Load validation dataset
# # # --------------------------------------------------
# # def load_jsonl(path):
# #     with open(path, "r") as f:
# #         return [json.loads(line) for line in f]

# # data_list = load_jsonl("valid.jsonl")
# # data_dict = {k: [d[k] for d in data_list] for k in data_list[0]}
# # valid_dataset = Dataset.from_dict(data_dict)

# # # --------------------------------------------------
# # # Transcribe & Compare
# # # --------------------------------------------------
# # def transcribe_and_compare(dataset):
# #     predictions, references, files = [], [], []

# #     for example in dataset:
# #         try:
# #             # 🔹 Load & resample audio to 16kHz
# #             waveform, sr = librosa.load(example["audio_filepath"], sr=16000)

# #             inputs = processor(
# #                 waveform, sampling_rate=16000, return_tensors="pt"
# #             )
# #             input_features = inputs.input_features

# #             with torch.no_grad():
# #                 predicted_ids = model.generate(input_features=input_features)

# #             pred_text = processor.batch_decode(
# #                 predicted_ids, skip_special_tokens=True
# #             )[0].strip()

# #             predictions.append(pred_text)
# #             references.append(example["human_transcript"].strip())
# #             files.append(example["audio_filepath"])

# #         except Exception as e:
# #             print(f"❌ Skipping {example['audio_filepath']}: {e}")

# #     if not predictions:
# #         print("⚠️ No valid transcriptions found.")
# #         return

# #     # Compute WER
# #     wer = wer_metric.compute(predictions=predictions, references=references)
# #     print(f"\n✅ Final Word Error Rate (WER): {wer:.3f}")

# #     # Side-by-side comparison
# #     df = pd.DataFrame({
# #         "Audio File": files,
# #         "Reference Transcript": references,
# #         "Model Transcript": predictions
# #     })

# #     from IPython.display import display
# #     display(df)

# #     return df


# # # --------------------------------------------------
# # # Run transcription
# # # --------------------------------------------------
# # df = transcribe_and_compare(valid_dataset)
# import json
# import torch
# import librosa
# import evaluate
# import pandas as pd
# from datasets import Dataset
# from transformers import WhisperProcessor, WhisperForConditionalGeneration

# # --------------------------------------------------
# # Load model and processor (choose large, large-v2, or large-v3)
# # --------------------------------------------------
# MODEL_PATH = "/home/sagemaker-user/1epoch_largelora_adapter"
# PROCESSOR_PATH = "/home/sagemaker-user/1epoch_largelora_fine_tuned_whisper"

# model = WhisperForConditionalGeneration.from_pretrained(MODEL_PATH)
# processor = WhisperProcessor.from_pretrained(PROCESSOR_PATH)

# # Disable forced language/decoder IDs (so predictions aren't biased)
# model.generation_config.forced_decoder_ids = None
# model.generation_config.suppress_tokens = []
# model.generation_config.language = "en"
# model.generation_config.task = "transcribe"
# model.eval()

# # --------------------------------------------------
# # Metric
# # --------------------------------------------------
# wer_metric = evaluate.load("wer")

# # --------------------------------------------------
# # Load validation dataset
# # --------------------------------------------------
# def load_jsonl(path: str):
#     """Load jsonl file with {'audio_filepath': ..., 'human_transcript': ...}"""
#     with open(path, "r", encoding="utf-8") as f:
#         return [json.loads(line) for line in f]

# data_list = load_jsonl("valid.jsonl")
# data_dict = {k: [d[k] for d in data_list] for k in data_list[0]}
# valid_dataset = Dataset.from_dict(data_dict)

# # --------------------------------------------------
# # Transcribe & Evaluate
# # --------------------------------------------------
# def transcribe_and_compare(dataset: Dataset):
#     predictions, references, files = [], [], []

#     for example in dataset:
#         try:
#             # Load & resample audio to 16kHz
#             waveform, _ = librosa.load(example["audio_filepath"], sr=16000)

#             # Prepare input features
#             inputs = processor(
#                 waveform, sampling_rate=16000, return_tensors="pt"
#             )
#             input_features = inputs.input_features

#             with torch.no_grad():
#                 predicted_ids = model.generate(
#                     input_features=input_features,
#                     max_length=448,   # prevent runaway generations
#                 )

#             pred_text = processor.batch_decode(
#                 predicted_ids, skip_special_tokens=True
#             )[0].strip()

#             predictions.append(pred_text)
#             references.append(example["human_transcript"].strip())
#             files.append(example["audio_filepath"])

#         except Exception as e:
#             print(f"❌ Skipping {example.get('audio_filepath', 'UNKNOWN')}: {e}")

#     if not predictions:
#         print("⚠️ No valid transcriptions found.")
#         return None

#     # Compute WER
#     wer = wer_metric.compute(predictions=predictions, references=references)
#     print(f"\n✅ Final Word Error Rate (WER): {wer:.3f}")

#     # Side-by-side comparison
#     df = pd.DataFrame({
#         "Audio File": files,
#         "Reference Transcript": references,
#         "Model Transcript": predictions
#     })

#     try:
#         from IPython.display import display
#         display(df)
#     except ImportError:
#         print(df.head())

#     return df


# # --------------------------------------------------
# # Run transcription
# # --------------------------------------------------
# if __name__ == "__main__":
#     df = transcribe_and_compare(valid_dataset)
# !pip install peft
import json
import torch
import librosa
import evaluate
import pandas as pd
from datasets import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel

# --------------------------------------------------
# Base model + LoRA adapter
# --------------------------------------------------
BASE_MODEL = "openai/whisper-large-v2"  # or "openai/whisper-large-v3"
ADAPTER_PATH = "3epochND_largev2lora_adapter"
PROCESSOR_PATH = "3epochND_largev2lora_fine_tuned_whisper"

# Load base model
model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)

# Attach LoRA adapter
model = PeftModel.from_pretrained(model, ADAPTER_PATH)

# Load processor (tokenizer + feature extractor)
processor = WhisperProcessor.from_pretrained(PROCESSOR_PATH)

# Generation config
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.generation_config.language = "en"
model.generation_config.task = "transcribe"
model.eval()


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1280, out_features=1280, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_featu

In [1]:
!pip install jiwer
import json
import torch
import librosa
import evaluate
import pandas as pd
from datasets import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel

# --------------------------------------------------
# Paths
# --------------------------------------------------
BASE_MODEL = "openai/whisper-large-v2"   # or "openai/whisper-large-v3"
ADAPTER_PATH = "3epochND_largev2lora_adapter"
PROCESSOR_PATH = "3epochND_largev2lora_fine_tuned_whisper"

# --------------------------------------------------
# Load model + processor
# --------------------------------------------------
print("🔹 Loading base Whisper model...")
model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)

print("🔹 Applying LoRA adapter...")
model = PeftModel.from_pretrained(model, ADAPTER_PATH)

print("🔹 Loading processor...")
processor = WhisperProcessor.from_pretrained(PROCESSOR_PATH)

# Disable forced decoder IDs (so predictions aren’t biased)
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.generation_config.language = "en"
model.generation_config.task = "transcribe"
model.eval()

# --------------------------------------------------
# Load WER metric
# --------------------------------------------------
wer_metric = evaluate.load("wer")

# --------------------------------------------------
# Load validation dataset
# --------------------------------------------------
def load_jsonl(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f]

data_list = load_jsonl("csv files/test.jsonl")
data_dict = {k: [d[k] for d in data_list] for k in data_list[0]}
valid_dataset = Dataset.from_dict(data_dict)

# --------------------------------------------------
# Transcribe & Compare
# --------------------------------------------------
# def transcribe_and_compare(dataset):
#     predictions, references, files = [], [], []

#     for example in dataset:
#         try:
#             # Load & resample audio to 16kHz
#             waveform, sr = librosa.load(example["audio_filepath"], sr=16000)

#             inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
#             input_features = inputs.input_features

#             with torch.no_grad():
#                 predicted_ids = model.generate(input_features=input_features)

#             pred_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

#             predictions.append(pred_text)
#             references.append(example["human_transcript"].strip())
#             files.append(example["audio_filepath"])

#         except Exception as e:
#             print(f"❌ Skipping {example['audio_filepath']}: {e}")

#     if not predictions:
#         print("⚠️ No valid transcriptions found.")
#         return

#     # Compute WER
#     wer = wer_metric.compute(predictions=predictions, references=references)
#     print(f"\n✅ Final Word Error Rate (WER): {wer:.3f}")

#     # Side-by-side comparison
#     df = pd.DataFrame({
#         "Audio File": files,
#         "Reference Transcript": references,
#         "Model Transcript": predictions
#     })

#     from IPython.display import display
#     display(df)

#     return df

# # --------------------------------------------------
# # Run transcription
# # --------------------------------------------------
# df = transcribe_and_compare(valid_dataset)
# --------------------------------------------------
# Transcribe & Compare
# --------------------------------------------------
def transcribe_and_compare(dataset, output_txt="transcriptions.txt"):
    predictions, references, files = [], [], []

    for example in dataset:
        try:
            # Load & resample audio to 16kHz
            waveform, sr = librosa.load(example["audio_filepath"], sr=16000)

            inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
            input_features = inputs.input_features

            with torch.no_grad():
                predicted_ids = model.generate(input_features=input_features)

            pred_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

            predictions.append(pred_text)
            references.append(example["human_transcript"].strip())
            files.append(example["audio_filepath"])

        except Exception as e:
            print(f"❌ Skipping {example['audio_filepath']}: {e}")

    if not predictions:
        print("⚠️ No valid transcriptions found.")
        return

    # Compute WER
    wer = wer_metric.compute(predictions=predictions, references=references)
    print(f"\n✅ Final Word Error Rate (WER): {wer:.3f}")

    # Save results to TXT file
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(f"Final WER: {wer:.3f}\n\n")
        f.write("Audio File\tReference Transcript\tModel Transcript\n")
        f.write("=" * 120 + "\n")
        for file, ref, pred in zip(files, references, predictions):
            f.write(f"{file}\nREF: {ref}\nPRED: {pred}\n\n")

    print(f"📂 Results saved to {output_txt}")
    return output_txt

# --------------------------------------------------
# Run transcription
# --------------------------------------------------
txt_file = transcribe_and_compare(valid_dataset)


2025-09-13 05:45:25.701990: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-13 05:45:25.718726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757742325.736144    2269 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757742325.741963    2269 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-13 05:45:25.759495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

🔹 Loading base Whisper model...
🔹 Applying LoRA adapter...
🔹 Loading processor...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



✅ Final Word Error Rate (WER): 6.164
📂 Results saved to transcriptions.txt


In [ ]:
# --------------------------------------------------
# Baseline: plain Whisper-large-v2
# --------------------------------------------------
print("🔹 Loading baseline Whisper model...")
baseline_model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
baseline_processor = WhisperProcessor.from_pretrained(BASE_MODEL)

baseline_model.generation_config.forced_decoder_ids = None
baseline_model.generation_config.suppress_tokens = []
baseline_model.generation_config.language = "en"
baseline_model.generation_config.task = "transcribe"
baseline_model.eval()

def transcribe_with_model(dataset, model, processor, name="Model"):
    predictions, references, files = [], [], []

    for example in dataset:
        try:
            waveform, sr = librosa.load(example["audio_filepath"], sr=16000)

            inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
            input_features = inputs.input_features

            with torch.no_grad():
                predicted_ids = model.generate(input_features=input_features)

            pred_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

            predictions.append(pred_text)
            references.append(example["human_transcript"].strip())
            files.append(example["audio_filepath"])

        except Exception as e:
            print(f"❌ Skipping {example['audio_filepath']}: {e}")

    if not predictions:
        print(f"⚠️ No valid transcriptions found for {name}.")
        return None

    wer = wer_metric.compute(predictions=predictions, references=references)
    print(f"\n✅ {name} Word Error Rate (WER): {wer:.3f}")

    return pd.DataFrame({
        "Audio File": files,
        f"{name} Transcript": predictions,
        "Reference Transcript": references
    })

# --------------------------------------------------
# Evaluate both models
# --------------------------------------------------
print("\n=== Fine-tuned LoRA Whisper ===")
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.generation_config.language = "en"
model.generation_config.task = "transcribe"

df_finetuned = transcribe_with_model(valid_dataset, model, processor, "Fine-tuned Whisper")

print("\n=== Baseline Whisper-large-v2 ===")
df_baseline = transcribe_with_model(valid_dataset, baseline_model, baseline_processor, "Baseline Whisper")

# --------------------------------------------------
# Merge results for side-by-side comparison
# --------------------------------------------------
if df_finetuned is not None and df_baseline is not None:
    df_compare = df_finetuned.merge(
        df_baseline[["Audio File", "Baseline Whisper Transcript"]],
        on="Audio File"
    )
    from IPython.display import display
    display(df_compare)


🔹 Loading baseline Whisper model...

=== Fine-tuned LoRA Whisper ===


In [ ]:
import librosa

# --------------------------------------------------
# Load a single test audio
# --------------------------------------------------
AUDIO_PATH = "data/SENT_0905.wav"
REFERENCE = "rendu Brownie, oru Akkaravadisal, anju Filter Coffee, naalu Cheesecake, moonu Cheese Corn Nuggets"

# Load audio at 16kHz
waveform, sr = librosa.load(AUDIO_PATH, sr=16000)

# --------------------------------------------------
# Transcription function
# --------------------------------------------------
def transcribe_single(model, processor, audio, reference=None, label=""):
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        predicted_ids = model.generate(input_features=inputs.input_features)
    pred_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

    print(f"\n🔹 {label} Prediction: {pred_text}")
    if reference:
        print(f"   📝 Reference: {reference}")
        wer = wer_metric.compute(predictions=[pred_text], references=[reference])
        print(f"   📊 WER: {wer:.3f}")

    return pred_text

# --------------------------------------------------
# 1. Base model (no LoRA)
# --------------------------------------------------
print("\n=== BASE MODEL ===")
base_model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
base_model.generation_config.forced_decoder_ids = None
base_model.generation_config.suppress_tokens = []
base_model.generation_config.language = "en"
base_model.generation_config.task = "transcribe"
base_model.eval()

transcribe_single(base_model, processor, waveform, REFERENCE, "Base")

# --------------------------------------------------
# 2. Fine-tuned model (with LoRA adapter)
# --------------------------------------------------
print("\n=== FINE-TUNED MODEL (LoRA) ===")
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.generation_config.language = "en"
model.generation_config.task = "transcribe"

transcribe_single(model, processor, waveform, REFERENCE, "Fine-tuned")
